In [26]:
import os
import math
from copy import deepcopy
import random
import numpy as np
import pysat
from pysat.solvers import Glucose3
import time


In [27]:
hx = [0, 1, 0, -1, -1, -1, 0, 1, 1]
hy = [0, -1, -1, -1, 0, 1, 1, 1, 0]

## read file

In [88]:
def readFile(filename):
    with open (filename, 'r') as f :
        s = f.readline()
        [m, n] = s.split(" ")
        [m, n] = [int(m), int(n)]

        Matrix =[]
        line = (f.readline()).split()
        while(line):
            Matrix.append([int(i) for i in line])
            line = (f.readline()).split()
    f.close()
    return [Matrix, m, n]
    

In [89]:
def gen_board_num(row,col):
    count=1
    matrix=[]

    for i in range(row):
        tmp=[]
        for j in range(col):
            tmp.append(count)
            count+=1
        matrix.append(tmp)

    return matrix

## generate & solve CNFs

In [90]:
# Tap hop R1, R2
def check_same_utility(R1,R2):
    if len(R1) != len(R2):
        return False
    R1 = sorted(deepcopy(R1))
    R2 = sorted(deepcopy(R2))
    for i in range(len(R2)):
        if R2[i] != R1[i]:
            return False
    return True

In [91]:
# To hop: C, Tap hop R
def check_same(C,R):
    if len(C) == 0 or len (R) == 0:
        return False
    for i in C:
        if check_same_utility(R,i):
            return True
    return False

In [92]:
def check_same_small(_list, num):
    if len(_list) == 0:
        return False
    for i in _list:
        if i == num:
            return True
    return False

In [93]:
def get_C(adjacentList, k, loop):
    C = []
    for _ in range(loop):
        tmp = []
        for _ in range(k):
            t = random.choice(adjacentList)
            while check_same_small(tmp,t):
                t = random.choice(adjacentList)
                
            tmp.append(t)
        while check_same(C,tmp):
            tmp = []
            for _ in range(k):
                t = random.choice(adjacentList)
                while check_same_small(tmp, t):
                    t = random.choice(adjacentList)
                tmp.append(t)
                
        C.append(tmp)
        
    return C

In [94]:
def get_adjacentList(i,j):
    res=[]

    if i-1 >= 0:
        res.append(board_num[i-1][j])
    if i+1 < len(a):
        res.append(board_num[i+1][j])
    if j-1 >= 0:
        res.append(board_num[i][j-1])
    if j+1 < len(a[0]):
        res.append(board_num[i][j+1])
    if i-1 >= 0 and j-1 >= 0:
        res.append(board_num[i-1][j-1])
    if i-1 >= 0 and j+1 < len(a[0]):
        res.append(board_num[i-1][j+1])
    if i+1 < len(a) and j-1 >= 0:
        res.append(board_num[i+1][j-1])
    if i+1 < len(a) and j+1 < len(a[0]):
        res.append(board_num[i+1][j+1])
    
    return res


In [95]:
def get_clauses(i, j):
    cell=board_num[i][j]
    adjacent_list=get_adjacentList(i,j)
    n=len(adjacent_list)
    k=a[i][j]
    
    clauses=[]
    # blue
    if k > 0 and (k <= n or (k-n) == 1):
        if k == 1:
            for e in adjacent_list:
                clauses.append([-cell,-e])
            tmp=deepcopy(adjacent_list)
            tmp.append(cell)
            clauses.append(tmp)
        else:
            loop_times=int(math.factorial(n)/(math.factorial(k-1)*math.factorial(n-k+1)))
            
            C=get_C(adjacent_list,k-1,loop_times)

            for R in C:
                # forward
                remain=[]
                for e in adjacent_list:
                    if not e in R:
                        remain.append(e)
                tmp=[]
                tmp.append(-cell)
                for m in R:
                    tmp.append(-m)
                for e in remain:
                    tmp_=deepcopy(tmp)
                    tmp_.append(-e)
                    clauses.append(tmp_)
                # reverse
                tmp_=deepcopy(remain)
                tmp_.append(cell)
                clauses.append(tmp_)

                for m in R:
                    tmp_=deepcopy(remain)
                    tmp_.append(m)
                    clauses.append(tmp_)

    #  red
    if k <= n:
        if k == 0:
            clauses.append([-cell])
            for e in adjacent_list:
                clauses.append([-e])
        else:  
            loop_times=int(math.factorial(n)/(math.factorial(k)*math.factorial(n-k)))
            C=get_C(adjacent_list, k,loop_times)
            for R in C:
                # forward
                remain=[]
                for e in adjacent_list:
                    if not e in R:
                        remain.append(e)
                tmp=[]
                for m in R:
                    tmp.append(-m)
                tmp_=deepcopy(tmp)
                tmp_.append(-cell)
                clauses.append(tmp_)

                for e in remain:
                    tmp_=deepcopy(tmp)
                    tmp_.append(-e)
                    clauses.append(tmp_)
                # reverse
                tmp_=deepcopy(remain)
                tmp_.append(cell)
                

                for m in R:
                    tmp__=deepcopy(tmp_)
                    tmp__.append(m)
                    clauses.append(tmp__)

    return clauses

## A*

In [96]:
def count_True_cell(clause, x, y):                    #Dem so o BLUE xung quang o(x,y)
    count = 0
    for k in range(9):
        tx = hx[k] + x
        ty = hy[k] + y
        if tx >= 0 and ty >= 0 and tx < m and ty < n:
            if clause[tx][ty] == True:
                count = count + 1

    return count


In [97]:
def is_Coloring_cell(x, y, b):        #Kiem tra xem o (x, y) to mau duoc hay khong?
    if a[x][y] == -1 and color[x][y] == True:
        return True

    tmp = deepcopy(b)

    for k in range(9):
        tx = hx[k] + x
        ty = hy[k] + y
        if (tx >= 0) and (ty >= 0) and (tx < m) and (ty < n) and (a[tx][ty] != -1):
            tmp[tx][ty] = tmp[tx][ty] - 1
            if tmp[tx][ty] < a[tx][ty]:
                return False
    return True

In [98]:
def is_Coloring_matrix(b):
    for i in range(m):
        for j in range(n):
            if a[i][j] != -1 and b[i][j] != a[i][j]:
                check = False
                for k in range(9):
                    tx = hx[k] + i
                    ty = hy[k] + j
                    if (tx >= 0) and (ty >= 0) and (tx < m) and (ty < n) and (color[tx][ty] == True):
                        if is_Coloring_cell(tx, ty, b) == True:
                            check = True
                            break
                if check == False:
                    return False
    return True


In [99]:
def printSolution(mark):
    for i in range(m):
        for j in range(n):
            if mark[i][j] == True:
                f.write("1 ")
            else:
                f.write("0 ")
        f.write('\n')

In [100]:
def count_Unsatisfied(current, goal):                     #Dem so o chua thoa man yeu cau
    c = 0
    for i in range(m):
        for j in range(n):
            if goal[i][j] != -1:
                if current[i][j] != goal[i][j]:
                    c = c + 1
    return c


In [101]:
def matrix_Satisfied(current):
    for i in range(m):
        for j in range(n):
            if a[i][j] != -1:
                if a[i][j] != current[i][j]:
                    check = False
                    for k in range(9):
                        tx = hx[k] + i
                        ty = hy[k] + j
                        if (tx >= 0) and (ty >= 0) and (tx < m) and (ty < n) and (color[tx][ty] == True):
                            if is_Coloring_cell(tx, ty, current) == True:
                                check = True
                                break
                    if check == False:
                        return False
    return True


In [102]:
def CountTotalCosttoGoal(current, goal):                    #Dem tong su chenh lech so voi goal
    c = 0
    if matrix_Satisfied(current) == False:
        return 10000
    for i in range(m):
        for j in range(n):
            if goal[i][j] != -1 and current[i][j] != goal[i][j]:
                c = c + (current[i][j] - goal[i][j])
    return c


In [103]:
def CountHofACell(x, y, start):                #Tinh heuristic cua o(x, y)
    if start[x][y] == a[x][y]:
        return 10000
    b = deepcopy(start)
    for k in range(9):
        tx = hx[k] + x
        ty = hy[k] + y
        if (tx >= 0 and ty >= 0 and tx < m and ty < n):
            b[tx][ty] = b[tx][ty] - 1
    
    count = count_Unsatisfied(b, a)
    count = count + CountTotalCosttoGoal(b, a)

    return count

In [104]:
def GenHeuristic(start, color):                  #Tinh heuristic va tim min
    h = deepcopy(start)
    for i in range(m):
        for j in range(n):
            if (start[i][j] == a[i][j]) or (is_Coloring_cell(i, j, start) == False):
                h[i][j] = 10000
            else:
                h[i][j] = CountHofACell(i, j, start)
    x = 0
    y = 0
    min = 10000000
    for i in range(m):
        for j in range(n):
            if h[i][j] < min and color[i][j] == True:
                min = h[i][j]
                x = i
                y = j
    
    return [x, y]


In [105]:
def EqualtoGoal(current, goal):
    for i in range(m):
        for j in range(n):
            if goal[i][j] != -1 and current[i][j] != goal[i][j]:
                return False

    return True

In [122]:
def AStar(start, color):
    cur_time = time.time()
    if (cur_time - start_time)/60 > 5:
        f.write("Run time is too large (> 10 minutes).")
        print("Run time is too large (> 10 minutes).")
        return -1
    for i in range(m):                                      #Dem so o mau xanh lien ke voi o (i,j)
        for j in range(n):
            start[i][j] = count_True_cell(color, i, j)

    if is_Coloring_matrix(start) == False:
        print("false")
        f.write("NO SOLUTION")
        return 1
    if EqualtoGoal(start, a) == False:
        [x, y] = GenHeuristic(start, color)
        color[x][y] = False
        AStar(start, color)
    else:
        printSolution(color)
        return 1

In [123]:
nameinp = "input4.txt"
nameoutp = "output2.txt"
f = open(nameoutp, 'w')
[a, m, n] = readFile(nameinp)
color = []
for i in range(m):
    li = []
    for j in range(n):
        li.append(True)
    color.append(li)

print("--------------------------------")
board_num=gen_board_num(m, n)
clauses=[]
for i in range(len(a)):
    for j in range(len(a[0])):
        if a[i][j] > -1:
            clauses+=get_clauses(i,j)
print("Number of CNFs generated: ", len(clauses))
f.write("Number of CNFs generated: ")
f.write(str(len(clauses)))
f.write('\n')

g = Glucose3()
for it in clauses:
    g.add_clause([int(k) for k in it])


kt = g.solve()
model = g.get_model()



f.write("\nUse A*:\n")
print(kt)
start_time = time.time()
if kt == True:
    start = deepcopy(a)
    AStar(start, color)
else:
    f.write("NO SOLUTION")
end_time = time.time()
print("A*:  total run-time: %f s" % ((end_time - start_time)))
print("--------------------------------")
f.close()

--------------------------------
Number of CNFs generated:  17589
True
false
A*:  total run-time: 21.354550 s
--------------------------------
